In [1]:
#-*- coding: utf-8 -*-
import os, gzip
import time
import tensorflow as tf
import numpy as np

import math
import random
import pprint
import scipy.misc
from six.moves import xrange

from tensorflow.python.framework import ops
import tensorflow.contrib.slim as slim

In [2]:
# 학습데이터 로드
def loadData(dataset_dir):
    print('INFO :: Loading Data...')
        
    # MNSIT_data 에 한해서만 구현, 나머지 추후구현
    if dataset_dir == 'MNIST_data':
        data_dir = os.path.join("./data", dataset_dir)

        def extract_data(filename, num_data, head_size, data_size):
            with gzip.open(filename) as bytestream:
                bytestream.read(head_size)
                buf = bytestream.read(data_size * num_data)
                data = np.frombuffer(buf, dtype=np.uint8).astype(np.float)
            return data

        data = extract_data(data_dir + '/train-images-idx3-ubyte.gz', 60000, 16, 28 * 28)
        trX = data.reshape((60000, 28, 28, 1))

        data = extract_data(data_dir + '/train-labels-idx1-ubyte.gz', 60000, 8, 1)
        trY = data.reshape((60000))

        data = extract_data(data_dir + '/t10k-images-idx3-ubyte.gz', 10000, 16, 28 * 28)
        teX = data.reshape((10000, 28, 28, 1))

        data = extract_data(data_dir + '/t10k-labels-idx1-ubyte.gz', 10000, 8, 1)
        teY = data.reshape((10000))

        trY = np.asarray(trY)
        teY = np.asarray(teY)

        X = np.concatenate((trX, teX), axis=0)
        y = np.concatenate((trY, teY), axis=0).astype(np.int)

        seed = 547
        np.random.seed(seed)
        np.random.shuffle(X)
        np.random.seed(seed)
        np.random.shuffle(y)

        y_vec = np.zeros((len(y), 10), dtype=np.float)
        for i, label in enumerate(y):
            y_vec[i, y[i]] = 1.0
            
        print('INFO :: MNIST_data - Load Complete!')
        
        return X / 255., y_vec
    else:
        raise NotImplementedError

In [3]:
"""ops and utils"""

if "concat_v2" in dir(tf):
    def concat(tensors, axis, *args, **kwargs):
        return tf.concat_v2(tensors, axis, *args, **kwargs)
else:
    def concat(tensors, axis, *args, **kwargs):
        return tf.concat(tensors, axis, *args, **kwargs)

def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

def conv2d(input_, output_dim, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
              initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')
        # params : input, filter, strides, padding

        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())
        return conv
    
def deconv2d(input_, output_shape, k_h=5, k_w=5, d_h=2, d_w=2, name="deconv2d", stddev=0.02, with_w=False):
    with tf.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))

        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape, strides=[1, d_h, d_w, 1])

        # Support for verisons of TensorFlow before 0.7.0
        except AttributeError:
            deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape, strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv, w, biases
        else:
            return deconv
    
def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                 tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
        initializer=tf.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias
        
def bn(x, is_training, scope):
    return tf.layers.batch_normalization(x, epsilon = 1e-5, momentum = 0.9, training = is_training, name = scope)


def show_all_variables():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

    
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    image = np.squeeze(merge(images, size))
    return scipy.misc.imsave(path, image)

def inverse_transform(images):
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    if (images.shape[3] in (3,4)):
        c = images.shape[3]
        img = np.zeros((h * size[0], w * size[1], c))
        for idx, image in enumerate(images):
            i = idx % size[1]
            j = idx // size[1]
            img[j * h:j * h + h, i * w:i * w + w, :] = image
        return img
    elif images.shape[3]==1:
        img = np.zeros((h * size[0], w * size[1]))
        for idx, image in enumerate(images):
            i = idx % size[1]
            j = idx // size[1]
            img[j * h:j * h + h, i * w:i * w + w] = image[:,:,0]
        return img
    else:
        raise ValueError('in merge(images,size) images parameter ''must have dimensions: HxW or HxWx3 or HxWx4')


In [4]:
class Settings(object):
    def __init__(self, sess, epoch, batch_size ,dataset_dir, checkpoint_dir, result_dir, log_dir, input_height, input_width, output_height, output_width, noise_vector_dim, result_vector_dim, c_dim, learning_rate, beta1, sample_test_num, len_discrete_code, len_continuous_code):
        
        self.sess = sess # tensorflow session
        self.epoch = epoch # epoch
        self.batch_size = batch_size # 배치 사이즈
        
        self.dataset_dir = dataset_dir # 데이터셋 디렉토리
        self.checkpoint_dir = checkpoint_dir # 체크포인트 디렉토리
        self.result_dir = result_dir # 결과 디렉토리
        self.log_dir = log_dir # 로그 디렉토리
        
        # parameters
        self.input_height = input_height # 입력 높이
        self.input_width = input_width # 입력 너비 
        self.output_height = output_height # 출력 높이
        self.output_width = output_width # 출력 너비
        
        self.noise_vector_dim = noise_vector_dim # 노이즈벡터 dimension
        self.result_vector_dim = result_vector_dim # 결과 벡터 dimension (label)
        self.c_dim = c_dim # c_dim ??
        
        # traing
        self.learning_rate = learning_rate # 러닝레이트
        self.beta1 = beta1 # beta1
        
        # test
        self.sample_test_num = sample_test_num # 샘플 테스트 갯수
        
        # code
        self.len_discrete_code = len_discrete_code # 범주별 분포 (label)
        self.len_continuous_code = len_continuous_code # 가우시안 분포 (ex. location, thickness...)
        
        # 데이터 로드, 배치 갯수 저장
        self.data_X, self.data_y = loadData(dataset_dir)
        self.num_batches = len(self.data_X) // batch_size
        

In [5]:
class ACGAN(object):
    def __init__(self, settings):
        self.settings = settings
        
    # discriminator
    # 이해하기 쉽게 하기 위해 전부 풀어썻습니다.
    def discriminator(self, x, is_training=True, reuse=False):
        with tf.variable_scope('discriminator', reuse=reuse):
            # layer 1 : convolution, relu
            print(' D Layer 1| initial x Shape: ',x.get_shape())
            d_net = conv2d(x, 64, 4, 4, 2, 2, name='d_conv1')
            print(' D Layer 1| d_net (after conv2d) Shape: ',d_net.get_shape())
            d_net = lrelu(d_net)
            print(' D Layer 1| d_net (after lrelu) Shape: ',d_net.get_shape())
            print('Finish D Layer 1|------------------------------------')
           
            # layer 2 : convolution, batch normalization, relu, reshape
            d_net = conv2d(d_net, 128, 4, 4, 2, 2, name='d_conv2')
            print(' D Layer 2| d_net (after conv2d) Shape: ', d_net.get_shape())
            d_net = bn(d_net, is_training, scope='d_bn2')
            print(' D Layer 2| d_net (after batch_normalization) Shape: ', d_net.get_shape())            
            d_net = lrelu(d_net)
            print(' D Layer 2| d_net (after lrelu) Shape: ',d_net.get_shape())
            d_net = tf.reshape(d_net, [self.settings.batch_size, -1])
            print(' D Layer 2| d_net (after reshape) Shape: ',d_net.get_shape())
            print('Finish D Layer 2|------------------------------------')
            
            # layer 3 : linear, bn, lrelu
            d_net = linear(d_net, 1024, scope='d_fc3')
            print(' D Layer 3| d_net (after linear) Shape: ',d_net.get_shape())
            d_net = bn(d_net, is_training, scope='d_bn3')
            print(' D Layer 3| d_net (after batch_normalization) Shape: ',d_net.get_shape())
            d_net = lrelu(d_net)
            print(' D Layer 3| d_net (after lrelu) Shape: ',d_net.get_shape())
            print('Finish D Layer 2|------------------------------------')
            
            # layer 4 : linear, sigmoid    
            out_logit = linear(d_net, 1, scope='d_fc4')
            print(' D Layer 4| out_logit (after linear) Shape: ',out_logit.get_shape())
            out = tf.nn.sigmoid(out_logit)
            print(' D Layer 4| out (after sigmoid) Shape: ',out.get_shape())
            print('Finish D Layer 4|------------------------------------')
            
            return out, out_logit, d_net
            
    # Generator
    # 이해하기 쉽게 하기 위해 전부 풀어썻습니다.
    def generator(self, noises, y, is_training=True, reuse=False):
        with tf.variable_scope('generator', reuse=reuse):
            # 노이즈와 코드를 merge
            noises = concat([noises, y], 1)
            print(' G Layer before| noises (after merge with  y) Shape: ',noises.get_shape())
            
            # layer 1 : linear, bn, relu
            g_net = linear(noises, 1024, scope='g_fc1')
            print(' G Layer 1| g_net (after linear) Shape: ',g_net.get_shape())
            g_net = bn(g_net, is_training, scope='g_bn1')
            print(' G Layer 1| g_net (after batch normalization) Shape: ',g_net.get_shape())
            g_net = tf.nn.relu(g_net)
            print(' G Layer 1| g_net (after relu) Shape: ',g_net.get_shape())
            print('Finish G Layer 1|------------------------------------')
            
            # layer 2 : linear, bn, relu, reshape
            g_net = linear(noises, 128*7*7, scope='g_fc2')
            print(' G Layer 2| g_net (after linear) Shape: ',g_net.get_shape())
            g_net = bn(g_net, is_training, scope='g_bn2')
            print(' G Layer 2| g_net (after batch normalization) Shape: ',g_net.get_shape())
            g_net = tf.nn.relu(g_net)
            print(' G Layer 2| g_net (after relu) Shape: ',g_net.get_shape())
            g_net = tf.reshape(g_net, [self.settings.batch_size, 7, 7, 128])
            print(' G Layer 2| g_net (after reshape) Shape: ',g_net.get_shape())
            print('Finish G Layer 2|------------------------------------')
            
            # layer 3 : deconv2d, bn, relu
            g_net = deconv2d(g_net, [self.settings.batch_size, 14, 14, 64], 4, 4, 2, 2, name='g_dc3')
            print(' G Layer 3| g_net (after deconv2d) Shape: ',g_net.get_shape())
            g_net = bn(g_net, is_training, scope='g_bn3')
            print(' G Layer 3| g_net (after g_bn3) Shape: ',g_net.get_shape())
            g_net = tf.nn.relu(g_net)
            print(' G Layer 3| g_net (after relu) Shape: ',g_net.get_shape())
            print('Finish G Layer 3|------------------------------------')
            
            # layer 4 : deconv2d, sigmoid
            g_net = deconv2d(g_net, [self.settings.batch_size, 28, 28, 1], 4, 4, 2, 2, name='g_dc4')
            print(' G Layer 4| g_net (after deconv2d) Shape: ',g_net.get_shape())
            out = tf.nn.sigmoid(g_net)
            print(' G Layer 4| out (after sigmoid) Shape: ',g_net.get_shape())
            print('Finish G Layer 4|------------------------------------')
            return out
            
    # TODO
    def classifier(self, x, is_training=True, reuse=False):
        with tf.variable_scope('classifier', reuse=reuse):
            net = lrelu(bn(linear(x, 128, scope='c_fc1'), is_training, scope='c_bn1'))
            out_logit = linear(net, self.settings.result_vector_dim, scope='c_fc2')
            out = tf.nn.softmax(out_logit)
            
            return out, out_logit
            
    def buildModel(self):
        settings = self.settings
        image_dims = [settings.input_height, settings.input_width, settings.c_dim] # input / output image dimension
        batch_size = settings.batch_size
        
        """ Graph 입력 """
        # 이미지
        self.input_images = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')
        # 라벨 (y)
        self.y = tf.placeholder(tf.float32, [batch_size, settings.result_vector_dim], name='labels')
        # 노이즈 (z)
        self.noises = tf.placeholder(tf.float32, [batch_size, settings.noise_vector_dim], name='noises')
        
        """ Loss Functions """
        ## 1. GAN Loss
        # 실제 이미지에 대한 Discriminator 의 출력
        D_real, D_real_logits, input4classifier_real = self.discriminator(self.input_images, is_training=True, reuse=False)
        
        # 가짜 이미지에 대한 Discriminator 의 출력
        G = self.generator(self.noises, self.y, is_training=True, reuse=False)
        D_fake, D_fake_logits, input4classifier_fake = self.discriminator(G, is_training=True, reuse=True)
        
        # Discriminator loss function
        d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
        d_loss_fake = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))

        self.d_loss = d_loss_real + d_loss_fake
        
        # Generator loss function
        self.g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))
        
        ## 2. Information Loss (?) 이해 잘안됨 TODO
        code_fake, code_logit_fake = self.classifier(input4classifier_fake, is_training=True, reuse=False)
        code_real, code_logit_real = self.classifier(input4classifier_real, is_training=True, reuse=True)
        
        # real sample information loss function
        q_real_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=code_logit_real, labels=self.y))
        
        # fake sample information loss function
        q_fake_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=code_logit_fake, labels=self.y))
        
        self.q_loss = q_fake_loss + q_real_loss
        
        """Training"""
        # 훈련 가능한 변수를 D에 대한 그룹과 G에 대한 그룹으로 나눔
        t_vars = tf.trainable_variables()
        d_vars = [var for var in t_vars if 'd_' in var.name]
        g_vars = [var for var in t_vars if 'g_' in var.name]
        q_vars = [var for var in t_vars if 'd_' or 'c_' or 'g_' in var.name]
        
        # Optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.d_optim = tf.train.AdamOptimizer(self.settings.learning_rate, beta1=self.settings.beta1) \
                .minimize(self.d_loss, var_list=d_vars)
            self.g_optim = tf.train.AdamOptimizer(self.settings.learning_rate * 5, beta1=self.settings.beta1) \
                .minimize(self.g_loss, var_list=g_vars)
            self.q_optim = tf.train.AdamOptimizer(self.settings.learning_rate * 5, beta1=self.settings.beta1) \
                .minimize(self.q_loss, var_list=q_vars)
        
        """Testing"""
        # for test
        self.fake_images = self.generator(self.noises, self.y, is_training=False, reuse=True)
        
        """ Summary """
        d_loss_real_sum = tf.summary.scalar("d_loss_real", d_loss_real)
        d_loss_fake_sum = tf.summary.scalar("d_loss_fake", d_loss_fake)
        d_loss_sum = tf.summary.scalar("d_loss", self.d_loss)
        g_loss_sum = tf.summary.scalar("g_loss", self.g_loss)

        q_loss_sum = tf.summary.scalar("g_loss", self.q_loss)
        q_real_sum = tf.summary.scalar("q_real_loss", q_real_loss)
        q_fake_sum = tf.summary.scalar("q_fake_loss", q_fake_loss)

        # final summary operations
        self.g_sum = tf.summary.merge([d_loss_fake_sum, g_loss_sum])
        self.d_sum = tf.summary.merge([d_loss_real_sum, d_loss_sum])
        self.q_sum = tf.summary.merge([q_loss_sum, q_real_sum, q_fake_sum])
        
    def train(self):
        # 모든 변수 초기화
        tf.global_variables_initializer().run()
    
        # 학습 결과를 비쥬얼라이징 하기 위한 그래프 입력
        self.sample_noises = np.random.uniform(-1, 1, size=(self.settings.batch_size, self.settings.noise_vector_dim))
        self.test_images = self.settings.data_X[0:self.settings.batch_size]
        self.test_labels = self.settings.data_y[0:self.settings.batch_size]
        self.test_codes = self.test_labels
        
        # 모델을 저장하기 위한 saver
        self.saver = tf.train.Saver()
        
        # summary writer
        self.writer = tf.summary.FileWriter(self.settings.log_dir + '/ACGAN', self.settings.sess.graph)

        # 체크포인트 디렉토리가가 있으면 restore check-point
        could_load, checkpoint_counter = self.load(self.settings.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_counter / self.settings.num_batches)
            start_batch_id = checkpoint_counter - start_epoch * self.settings.num_batches
            counter = checkpoint_counter
            print(" [*] Load SUCCESS")
        else:
            start_epoch = 0
            start_batch_id = 0
            counter = 1
            print(" [!] Load failed...")
        
        # 학습!
        start_time = time.time()
        for epoch in range(start_epoch, self.settings.epoch):
            # 배치 데이터 가져옴
            for idx in range(start_batch_id, self.settings.num_batches):
                batch_images = self.settings.data_X[idx*self.settings.batch_size:(idx+1)*self.settings.batch_size]
                batch_labels = self.settings.data_y[idx*self.settings.batch_size:(idx+1)*self.settings.batch_size]
                batch_codes = batch_labels
                
                batch_noises = np.random.uniform(-1, 1, [self.settings.batch_size, self.settings.noise_vector_dim]).astype(np.float32)
                
                # update D network
                _, summary_str, d_loss = self.settings.sess.run([self.d_optim, self.d_sum, self.d_loss],
                                                       feed_dict={self.input_images: batch_images, self.y: batch_codes,
                                                                  self.noises: batch_noises})
                self.writer.add_summary(summary_str, counter)
                
                # update G network
                _, summary_str_g, g_loss, _, summary_str_q, q_loss = self.settings.sess.run(
                    [self.g_optim, self.g_sum, self.g_loss, self.q_optim, self.q_sum, self.q_loss],
                    feed_dict={self.input_images: batch_images, self.noises: batch_noises, self.y: batch_codes})
                self.writer.add_summary(summary_str_g, counter)
                self.writer.add_summary(summary_str_q, counter)
                
                 # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f" \
                      % (epoch, idx, self.settings.num_batches, time.time() - start_time, d_loss, g_loss))
                
                # 100 스탭마다 훈련한 이미지를 저장
                if np.mod(counter, 100) == 0:
                    samples = self.settings.sess.run(self.fake_images,
                                            feed_dict={self.noises: self.sample_noises, self.input_images: self.test_images, self.y: self.test_codes})
                    tot_num_samples = min(self.settings.sample_test_num, self.settings.batch_size)
                    manifold_h = int(np.floor(np.sqrt(tot_num_samples)))
                    manifold_w = int(np.floor(np.sqrt(tot_num_samples)))
                    save_images(samples[:manifold_h*manifold_w,:,:,:], [manifold_h, manifold_w],
                          './'+self.settings.result_dir+'/'+'ACGAN'+'_train_{:02d}_{:04d}.png'.format(epoch, idx))

            # epoch 한번 돌면 start_batch_id 초기화
            start_batch_id = 0
                
            # save model
            self.save(self.settings.checkpoint_dir, counter)

            # show temporal results
            # self.visualize_results(epoch)

        # save model for final step
        self.save(self.settings.checkpoint_dir, counter)
        
    
    @property
    def model_dir(self):
        return "ACGAN_{}_{}_{}".format(
            self.settings.batch_size,
            self.settings.output_height, self.settings.output_width)
        
    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, 'ACGAN')

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.settings.sess,os.path.join(checkpoint_dir, 'ACGAN.model'), global_step=step)
        
    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, 'ACGAN')

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.settings.sess, os.path.join(checkpoint_dir, ckpt_name))
            counter = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read {}".format(ckpt_name))
            return True, counter
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0

In [6]:
def main():
    # 커스텀 변수
    epoch = 10 # epoch
    batch_size = 64 # 배치 사이즈
    dataset_dir = 'MNIST_data' # 데이터셋 디렉토리
    checkpoint_dir = 'checkpoints' # 체크포인트 디렉토리
    result_dir = 'results' # 결과 디렉토리
    log_dir = 'logs' # 로그 디렉토리
    
    # parameters
    input_height = 28 # 입력 높이
    input_width = 28 # 입력 너비 
    output_height = 28 # 출력 높이
    output_width = 28 # 출력 너비
    noise_vector_dim = 62 # 노이즈벡터 dimension
    result_vector_dim = 10 # 코드 벡터 dimension (label)
    c_dim = 1 # c_dim ??
    
    # traing
    learning_rate = 0.0005 # 러닝레이트
    beta1 = 0.5 # beta1
    
    # test
    sample_test_num = 64 # 샘플 테스트 갯수
    
    # code
    len_discrete_code = 10 # 범주별 분포 (label)
    len_continuous_code = 2 # 가우시안 분포 (ex. location, thickness...)
    
    
    
    # 그래프를 launch 하기 위해서는 session 을 생성해주어야함
    with tf.Session() as sess:
        
        # 세팅 주입
        settings = Settings(sess, epoch, batch_size ,dataset_dir, checkpoint_dir, result_dir,log_dir, input_height, input_width, output_height, output_width, noise_vector_dim, result_vector_dim, c_dim, learning_rate, beta1, sample_test_num, len_discrete_code, len_continuous_code)
    
        acgan = ACGAN(settings)
        acgan.buildModel()
        show_all_variables()
        acgan.train()

if __name__ == '__main__':
    main()

INFO :: Loading Data...
INFO :: MNIST_data - Load Complete!
 D Layer 1| initial x Shape:  (64, 28, 28, 1)
 D Layer 1| d_net (after conv2d) Shape:  (64, 14, 14, 64)
 D Layer 1| d_net (after lrelu) Shape:  (64, 14, 14, 64)
Finish D Layer 1|------------------------------------
 D Layer 2| d_net (after conv2d) Shape:  (64, 7, 7, 128)
 D Layer 2| d_net (after batch_normalization) Shape:  (64, 7, 7, 128)
 D Layer 2| d_net (after lrelu) Shape:  (64, 7, 7, 128)
 D Layer 2| d_net (after reshape) Shape:  (64, 6272)
Finish D Layer 2|------------------------------------
 D Layer 3| d_net (after linear) Shape:  (64, 1024)
 D Layer 3| d_net (after batch_normalization) Shape:  (64, 1024)
 D Layer 3| d_net (after lrelu) Shape:  (64, 1024)
Finish D Layer 2|------------------------------------
 D Layer 4| out_logit (after linear) Shape:  (64, 1)
 D Layer 4| out (after sigmoid) Shape:  (64, 1)
Finish D Layer 4|------------------------------------
 G Layer before| noises (after merge with  y) Shape:  (64,

Epoch: [ 1] [  28/1093] time: 21.6068, d_loss: 1.29445195, g_loss: 0.79132330
Epoch: [ 1] [  29/1093] time: 22.3548, d_loss: 1.31118751, g_loss: 0.81498992
Epoch: [ 1] [  30/1093] time: 23.1038, d_loss: 1.27862108, g_loss: 0.83528721
Epoch: [ 1] [  31/1093] time: 23.9025, d_loss: 1.30156016, g_loss: 0.83363932
Epoch: [ 1] [  32/1093] time: 24.6509, d_loss: 1.31775999, g_loss: 0.77483618
Epoch: [ 1] [  33/1093] time: 25.4250, d_loss: 1.30621624, g_loss: 0.76293975
Epoch: [ 1] [  34/1093] time: 26.1779, d_loss: 1.30122113, g_loss: 0.78760195
Epoch: [ 1] [  35/1093] time: 26.9331, d_loss: 1.29013228, g_loss: 0.81198800
Epoch: [ 1] [  36/1093] time: 27.7084, d_loss: 1.29071212, g_loss: 0.82453358
Epoch: [ 1] [  37/1093] time: 28.4507, d_loss: 1.33782768, g_loss: 0.81185293
Epoch: [ 1] [  38/1093] time: 29.1873, d_loss: 1.30891895, g_loss: 0.80789030
Epoch: [ 1] [  39/1093] time: 29.9225, d_loss: 1.34173822, g_loss: 0.77285397
Epoch: [ 1] [  40/1093] time: 30.6826, d_loss: 1.29614806, g_los

Epoch: [ 1] [ 134/1093] time: 100.1440, d_loss: 1.32568789, g_loss: 0.77500242
Epoch: [ 1] [ 135/1093] time: 100.9119, d_loss: 1.30924892, g_loss: 0.80269957
Epoch: [ 1] [ 136/1093] time: 101.6618, d_loss: 1.29411316, g_loss: 0.81184071
Epoch: [ 1] [ 137/1093] time: 102.4063, d_loss: 1.30863190, g_loss: 0.78884202
Epoch: [ 1] [ 138/1093] time: 103.1859, d_loss: 1.33048058, g_loss: 0.77767849
Epoch: [ 1] [ 139/1093] time: 104.0215, d_loss: 1.32777786, g_loss: 0.76383692
Epoch: [ 1] [ 140/1093] time: 104.8230, d_loss: 1.33990145, g_loss: 0.76083839
Epoch: [ 1] [ 141/1093] time: 105.5671, d_loss: 1.29179430, g_loss: 0.81864351
Epoch: [ 1] [ 142/1093] time: 106.3161, d_loss: 1.31925905, g_loss: 0.78169674
Epoch: [ 1] [ 143/1093] time: 107.0714, d_loss: 1.29156160, g_loss: 0.80075192
Epoch: [ 1] [ 144/1093] time: 107.8211, d_loss: 1.28516448, g_loss: 0.79200655
Epoch: [ 1] [ 145/1093] time: 108.5877, d_loss: 1.30372787, g_loss: 0.77843958
Epoch: [ 1] [ 146/1093] time: 109.3345, d_loss: 1.27

Epoch: [ 1] [ 238/1093] time: 180.3773, d_loss: 1.29368770, g_loss: 0.79790479
Epoch: [ 1] [ 239/1093] time: 181.1699, d_loss: 1.30840838, g_loss: 0.78196788
Epoch: [ 1] [ 240/1093] time: 181.9502, d_loss: 1.26185429, g_loss: 0.77676582
Epoch: [ 1] [ 241/1093] time: 182.7125, d_loss: 1.32581401, g_loss: 0.78274381
Epoch: [ 1] [ 242/1093] time: 183.4834, d_loss: 1.31280351, g_loss: 0.78767049
Epoch: [ 1] [ 243/1093] time: 184.2518, d_loss: 1.30707860, g_loss: 0.77549982
Epoch: [ 1] [ 244/1093] time: 185.0243, d_loss: 1.31900167, g_loss: 0.78646320
Epoch: [ 1] [ 245/1093] time: 185.7939, d_loss: 1.29365981, g_loss: 0.77985770
Epoch: [ 1] [ 246/1093] time: 186.5624, d_loss: 1.29223895, g_loss: 0.78579891
Epoch: [ 1] [ 247/1093] time: 187.3429, d_loss: 1.32744002, g_loss: 0.77665865
Epoch: [ 1] [ 248/1093] time: 188.1165, d_loss: 1.34306335, g_loss: 0.75311029
Epoch: [ 1] [ 249/1093] time: 188.9050, d_loss: 1.29907465, g_loss: 0.78483045
Epoch: [ 1] [ 250/1093] time: 189.6831, d_loss: 1.30

Epoch: [ 1] [ 342/1093] time: 262.5979, d_loss: 1.25707889, g_loss: 0.82503021
Epoch: [ 1] [ 343/1093] time: 263.3909, d_loss: 1.31615806, g_loss: 0.76414055
Epoch: [ 1] [ 344/1093] time: 264.1697, d_loss: 1.30072379, g_loss: 0.78350013
Epoch: [ 1] [ 345/1093] time: 264.9567, d_loss: 1.28473556, g_loss: 0.81075245
Epoch: [ 1] [ 346/1093] time: 265.7206, d_loss: 1.31616080, g_loss: 0.78677666
Epoch: [ 1] [ 347/1093] time: 266.5115, d_loss: 1.30290687, g_loss: 0.78912663
Epoch: [ 1] [ 348/1093] time: 267.2820, d_loss: 1.33762312, g_loss: 0.75164431
Epoch: [ 1] [ 349/1093] time: 268.0583, d_loss: 1.29287052, g_loss: 0.77018088
Epoch: [ 1] [ 350/1093] time: 268.8532, d_loss: 1.29695964, g_loss: 0.78944647
Epoch: [ 1] [ 351/1093] time: 269.6289, d_loss: 1.30330729, g_loss: 0.81007594
Epoch: [ 1] [ 352/1093] time: 270.3942, d_loss: 1.29484344, g_loss: 0.80239773
Epoch: [ 1] [ 353/1093] time: 271.1952, d_loss: 1.28073168, g_loss: 0.78958023
Epoch: [ 1] [ 354/1093] time: 271.9908, d_loss: 1.28

...

Epoch: [ 9] [ 671/1093] time: 7155.4645, d_loss: 1.23069096, g_loss: 0.85105544
Epoch: [ 9] [ 672/1093] time: 7156.2385, d_loss: 1.28479481, g_loss: 0.80498928
Epoch: [ 9] [ 673/1093] time: 7157.0326, d_loss: 1.25033760, g_loss: 0.86692226
Epoch: [ 9] [ 674/1093] time: 7157.8440, d_loss: 1.28719175, g_loss: 0.85109484
Epoch: [ 9] [ 675/1093] time: 7158.6245, d_loss: 1.25614297, g_loss: 0.83931243
Epoch: [ 9] [ 676/1093] time: 7159.4408, d_loss: 1.28307986, g_loss: 0.85374779
Epoch: [ 9] [ 677/1093] time: 7160.2193, d_loss: 1.28782606, g_loss: 0.85176688
Epoch: [ 9] [ 678/1093] time: 7161.0026, d_loss: 1.21951294, g_loss: 0.86905372
Epoch: [ 9] [ 679/1093] time: 7161.7740, d_loss: 1.28699780, g_loss: 0.83881354
Epoch: [ 9] [ 680/1093] time: 7162.5274, d_loss: 1.32543993, g_loss: 0.81387901
Epoch: [ 9] [ 681/1093] time: 7163.3053, d_loss: 1.31217790, g_loss: 0.81270987
Epoch: [ 9] [ 682/1093] time: 7164.0582, d_loss: 1.21477783, g_loss: 0.85397607
Epoch: [ 9] [ 683/1093] time: 7164.8195,

Epoch: [ 9] [ 774/1093] time: 7234.1101, d_loss: 1.28673851, g_loss: 0.87876159
Epoch: [ 9] [ 775/1093] time: 7234.8654, d_loss: 1.30529678, g_loss: 0.81754905
Epoch: [ 9] [ 776/1093] time: 7235.6103, d_loss: 1.23395014, g_loss: 0.84030390
Epoch: [ 9] [ 777/1093] time: 7236.3664, d_loss: 1.24825454, g_loss: 0.82126737
Epoch: [ 9] [ 778/1093] time: 7237.1266, d_loss: 1.30328488, g_loss: 0.81522828
Epoch: [ 9] [ 779/1093] time: 7237.8528, d_loss: 1.28978860, g_loss: 0.81536901
Epoch: [ 9] [ 780/1093] time: 7238.6053, d_loss: 1.23859310, g_loss: 0.85780299
Epoch: [ 9] [ 781/1093] time: 7239.3671, d_loss: 1.25239527, g_loss: 0.87736797
Epoch: [ 9] [ 782/1093] time: 7240.1206, d_loss: 1.26324320, g_loss: 0.84136677
Epoch: [ 9] [ 783/1093] time: 7240.8483, d_loss: 1.23984170, g_loss: 0.83852136
Epoch: [ 9] [ 784/1093] time: 7241.6112, d_loss: 1.29296362, g_loss: 0.79946369
Epoch: [ 9] [ 785/1093] time: 7242.3641, d_loss: 1.28259885, g_loss: 0.81308722
Epoch: [ 9] [ 786/1093] time: 7243.1452,

Epoch: [ 9] [ 877/1093] time: 7313.0592, d_loss: 1.25925601, g_loss: 0.86905795
Epoch: [ 9] [ 878/1093] time: 7313.8057, d_loss: 1.29056823, g_loss: 0.84534872
Epoch: [ 9] [ 879/1093] time: 7314.5641, d_loss: 1.21635103, g_loss: 0.85248923
Epoch: [ 9] [ 880/1093] time: 7315.3188, d_loss: 1.26753426, g_loss: 0.84405267
Epoch: [ 9] [ 881/1093] time: 7316.0895, d_loss: 1.28338122, g_loss: 0.81614459
Epoch: [ 9] [ 882/1093] time: 7316.8602, d_loss: 1.27332187, g_loss: 0.83234948
Epoch: [ 9] [ 883/1093] time: 7317.6188, d_loss: 1.25982547, g_loss: 0.82354879
Epoch: [ 9] [ 884/1093] time: 7318.4392, d_loss: 1.27178454, g_loss: 0.84231323
Epoch: [ 9] [ 885/1093] time: 7319.2430, d_loss: 1.29143536, g_loss: 0.80917263
Epoch: [ 9] [ 886/1093] time: 7320.0379, d_loss: 1.23259270, g_loss: 0.82484758
Epoch: [ 9] [ 887/1093] time: 7320.8068, d_loss: 1.23489404, g_loss: 0.83589804
Epoch: [ 9] [ 888/1093] time: 7321.6211, d_loss: 1.30849349, g_loss: 0.87011468
Epoch: [ 9] [ 889/1093] time: 7322.3764,

Epoch: [ 9] [ 980/1093] time: 7393.5980, d_loss: 1.21619689, g_loss: 0.87680233
Epoch: [ 9] [ 981/1093] time: 7394.3794, d_loss: 1.30454183, g_loss: 0.80660713
Epoch: [ 9] [ 982/1093] time: 7395.1544, d_loss: 1.24575603, g_loss: 0.84021217
Epoch: [ 9] [ 983/1093] time: 7395.9338, d_loss: 1.27469814, g_loss: 0.86770225
Epoch: [ 9] [ 984/1093] time: 7396.7148, d_loss: 1.23221874, g_loss: 0.83787924
Epoch: [ 9] [ 985/1093] time: 7397.4960, d_loss: 1.28792143, g_loss: 0.84115446
Epoch: [ 9] [ 986/1093] time: 7398.2893, d_loss: 1.27996254, g_loss: 0.82778156
Epoch: [ 9] [ 987/1093] time: 7399.1121, d_loss: 1.29725850, g_loss: 0.80461836
Epoch: [ 9] [ 988/1093] time: 7399.8596, d_loss: 1.25088370, g_loss: 0.87351859
Epoch: [ 9] [ 989/1093] time: 7400.6036, d_loss: 1.22979748, g_loss: 0.87092060
Epoch: [ 9] [ 990/1093] time: 7401.3488, d_loss: 1.20378828, g_loss: 0.84787655
Epoch: [ 9] [ 991/1093] time: 7402.1284, d_loss: 1.25265980, g_loss: 0.84513110
Epoch: [ 9] [ 992/1093] time: 7402.9091,

Epoch: [ 9] [1083/1093] time: 7472.7649, d_loss: 1.26834130, g_loss: 0.82285917
Epoch: [ 9] [1084/1093] time: 7473.4869, d_loss: 1.16317940, g_loss: 0.89162123
Epoch: [ 9] [1085/1093] time: 7474.2339, d_loss: 1.28201199, g_loss: 0.85585785
Epoch: [ 9] [1086/1093] time: 7475.0036, d_loss: 1.35139215, g_loss: 0.81661737
Epoch: [ 9] [1087/1093] time: 7475.7593, d_loss: 1.28700233, g_loss: 0.78710735
Epoch: [ 9] [1088/1093] time: 7476.5351, d_loss: 1.32665884, g_loss: 0.77683562
Epoch: [ 9] [1089/1093] time: 7477.2822, d_loss: 1.23848093, g_loss: 0.85007191
Epoch: [ 9] [1090/1093] time: 7478.0424, d_loss: 1.28342271, g_loss: 0.79848039
Epoch: [ 9] [1091/1093] time: 7478.8606, d_loss: 1.27393687, g_loss: 0.84415323
Epoch: [ 9] [1092/1093] time: 7479.6384, d_loss: 1.20194626, g_loss: 0.87892377
